In [1]:
import sys
sys.path.append("../")

In [32]:
import os
from data import get_ute_query_loader
from data.ute_shot_query import get_ute_shot_query_loader
from utils.semantic_evaluation import load_videos_tag, semantic_iou
import random
import json
import numpy as np
import networkx as nx
from sklearn.metrics import pairwise_distances


In [10]:
class DataConfig:
    def __init__(self):
        self.annotation_dir = "/mnt/d/Data/UTE/UTC_feature/data/origin_data"
        self.dictionary_path = "/mnt/d/Data/UTE/UTC_feature/data/processed/query_dictionary.pkl"
        self.feature_dir = "/mnt/d/Data/UTE/UTC_feature/data/processed"
        self.batch_size = 4

config = DataConfig()
videos_tag = load_videos_tag("../data/ute_query/Tags.mat")

In [94]:
def build_gt_summaries(videos, annotation_dir):
    gt_summaries = {}
    oracle_summaries = []
    oracle_summary_dir = os.path.join(config.annotation_dir, "Query-Focused_Summaries/Oracle_Summaries")
    for video_id in videos:
        video_dir = os.path.join(oracle_summary_dir,  str(video_id))
        for _, _, files in os.walk(video_dir):
            for file in files:
                gt_summary = []
                record_entry = file[:file.find("_oracle.txt")] + "_" + str(video_id)
                oracle_summaries.append(record_entry)
                with open(os.path.join(video_dir, file),
                          "r") as f:
                    for line in f.readlines():
                        gt_summary.append(int(line.strip()))
                gt_summaries[record_entry] = gt_summary
    # print(oracle_summaries)
    shot_tag_dir = os.path.join(config.annotation_dir, "Dense_per_shot_tags")
    return oracle_summaries, gt_summaries


def build_graph_from_weights(weight_matrix):
    G = nx.Graph()
    nodes = list(map(lambda x: "n-{}".format(x), list(range(weight_matrix.shape[0]))))
    edges = []
    for i in range(weight_matrix.shape[0]):
        for j in range(i+1, weight_matrix.shape[1]):
            weight = weight_matrix[i][j]
            edges.append(("n-{}".format(i), "n-{}".format(j), weight))
    G.add_weighted_edges_from(edges)
    return G



def select_shots(shots, video_tags, target_len=5):
    # Processing shot list
    shots = np.array(shots)
    shots = shots - 1
    
    # Build the graph
    tag_vectors = video_tags[shots]
    weights = pairwise_distances(tag_vectors, tag_vectors, metric=semantic_iou)
    G = build_graph_from_weights(weights)
    
    # Calculate the centrality
    centrality = nx.eigenvector_centrality(G, weight="weight", max_iter=600)
    centrality_values = list(map(lambda x: centrality["n-{}".format(x)], list(range(shots.shape[0]))))
    
    # Get the top shots
    sorted_indices = np.argsort(centrality_values)
    sorted_indices = sorted_indices[::-1]
    selected_indices = sorted_indices[:target_len]
    selected_shots = shots[selected_indices]
    selected_shots = list(map(lambda x: int(x), selected_shots))
    centrality_values = np.array(centrality_values)
    print(selected_shots)
    # print(centrality_values[selected_indices])
    # selected_shots = list(map(lambda x: int(x), selected_shots))
    return selected_shots


def build_query_shot_index(videos, annotation_dir, videos_tag):
    keys, gt_summaries = build_gt_summaries(videos, config.annotation_dir)
    query_shot_index = {}
    for key in keys:
        video_id = key.split("_")[2]
        video_tags = videos_tag[int(video_id[1:]) - 1]
        query_shots = select_shots(gt_summaries[key], video_tags)
        query_shot_index[key] = query_shots
    return query_shot_index


def build_query_shot_file(videos, annotation_dir, output_path, tag_path="../data/ute_query/Tags.mat"):
    videos_tag = load_videos_tag(tag_path)
    query_shot_index = build_query_shot_index(videos, annotation_dir, videos_tag)
    
    with open(output_path, "w") as fp:
        json.dump(query_shot_index, fp)

build_query_shot_file(["P01", "P02", "P03", "P04"], config.annotation_dir, "../data/ute_query/shot_query.json")

[549, 1520, 593, 1375, 1369]
[2603, 53, 1673, 2031, 2449]
[2603, 2031, 975, 827, 1518]
[1520, 593, 1510, 1518, 502]
[549, 554, 562, 593, 581]
[2532, 794, 1613, 868, 613]
[1613, 1677, 845, 1616, 622]
[622, 593, 693, 613, 1326]
[2379, 2647, 1908, 2003, 2005]
[1616, 218, 1861, 1854, 1694]
[466, 441, 125, 97, 2207]
[593, 509, 1326, 1533, 520]
[159, 810, 794, 1069, 315]
[47, 42, 45, 46, 2099]
[341, 24, 794, 877, 70]
[497, 3, 1326, 1688, 593]
[208, 1326, 841, 821, 1002]
[1928, 1916, 2013, 2008, 2007]
[146, 407, 118, 124, 126]
[765, 1, 50, 78, 2425]
[124, 118, 156, 126, 146]
[1637, 1638, 1721, 1644, 492]
[1619, 136, 1787, 1655, 493]
[208, 712, 690, 151, 2059]
[6, 1562, 10, 1526, 1770]
[593, 1326, 1369, 2073, 1724]
[1840, 492, 462, 928, 7]
[845, 441, 1086, 279, 262]
[208, 3, 1688, 1326, 151]
[1637, 8, 1825, 1721, 490]
[8, 4, 597, 1617, 490]
[119, 2073, 467, 6, 255]
[1532, 1889, 765, 1326, 605]
[549, 593, 1365, 581, 1375]
[320, 704, 705, 708, 1341]
[1968, 1913, 1936, 2073, 868]
[593, 1326, 1688

In [91]:
train_loaders = get_ute_shot_query_loader(["P01", "P02", "P03", "P04"], config, shuffle=True, drop_last=True)

In [6]:
for batch in train_loaders:
    print(batch)

TypeError: 'int' object is not callable

[array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 1, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 array([[0, 0, 0, ..., 0, 0, 1],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 1],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 array([[0, 0, 0, ..., 0, 0, 1],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 1]], dtype=uint8)]